In [ ]:
# Kiểm tra setup
try:
    from shared_variables import *
except:
    # Tải các biến và hàm từ notebook trước
    %run "1_Setup_and_Utils.ipynb"
    %run "2_Image_Processing.ipynb"
    %run "3_Model_Definition.ipynb"

# Load các hàm xử lý ảnh
extract_hand_region = load_variable('extract_hand_region')
convert_to_grayscale = load_variable('convert_to_grayscale')
draw_prediction = load_variable('draw_prediction')

# Load model class và model đã huấn luyện
SignLanguageModel = load_variable('SignLanguageModel')
trained_model = load_variable('trained_model')

# Hàm dự đoán với ảnh được upload
def predict_with_uploaded_image():
    """Predict sign using an uploaded image"""
    print("Upload an image of a hand sign to test the model")
    uploaded = files.upload()
    
    if not uploaded:
        print("No image uploaded")
        return
    
    # Load the model
    if os.path.exists("models/sign_model.h5"):
        model = SignLanguageModel("models/sign_model.h5")
    elif trained_model is not None:
        model = trained_model
    else:
        print("No trained model found. Please train or upload a model first.")
        return
    
    # Process each uploaded image
    for filename, content in uploaded.items():
        image = cv2.imdecode(np.frombuffer(content, np.uint8), cv2.IMREAD_COLOR)
        
        # Extract hand region
        hand_region = extract_hand_region(image.copy())
        
        # Convert to grayscale
        gray_image = convert_to_grayscale(hand_region)
        
        # Get prediction
        sign, confidence = model.predict_sign(gray_image)
        
        # Apply confidence threshold
        if confidence < 0.4:
            sign = "?"
        
        # Draw prediction on image
        result = draw_prediction(image, sign, confidence)
        
        # Display result
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        plt.title(f"Prediction: {sign} (Confidence: {confidence:.2f})")
        plt.axis('off')
        plt.show()

# Hàm upload mô hình
def upload_model():
    """Upload a pre-trained model"""
    print("Upload a pre-trained model file (.h5)")
    uploaded = files.upload()
    
    if not uploaded:
        return False
    
    for filename in uploaded.keys():
        if filename.endswith('.h5'):
            # Tạo thư mục models nếu chưa có
            if not os.path.exists('models'):
                os.makedirs('models')
                
            # Save the uploaded model
            with open("models/sign_model.h5", 'wb') as f:
                f.write(uploaded[filename])
            
            print(f"Model uploaded as models/sign_model.h5")
            
            # Tạo instance với model đã upload
            model = SignLanguageModel("models/sign_model.h5")
            save_variable(model, 'trained_model')
            
            return True
    
    print("No valid model file uploaded. Please upload a .h5 file.")
    return False

# Menu chính
print("# Sign Language Recognition - Test & Predict")
print("\nPlease choose an option:")
print("1. Upload a model")
print("2. Test with uploaded image")
print("3. Exit")

choice = input("Enter your choice (1-3): ")

if choice == '1':
    upload_model()
elif choice == '2':
    predict_with_uploaded_image()
else:
    print("Thank you for using the Sign Language Recognition system.")